In [1]:
#%% Merging and Cleaning iSLims data
# Garrett

# Packages
import pandas as pd

In [2]:
# Loading data
wo = pd.read_excel('C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data/islims_workorders.xlsx')
de = pd.read_excel('C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data/islims_workorders_detail.xlsx')
iv = pd.read_excel('C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data/islims_inventory.xlsx')
fc = pd.read_excel('C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data/islims_failure_codes.xlsx')

In [183]:
# Merging work order and details sets on woID
wode = pd.merge(wo, de, on='woID')

In [184]:
# Merging wode and inventory sets on inventoryID
iSlimsa = pd.merge(wode, iv, on='inventoryID')

In [185]:
# Merging the GPS coordinates to fill in as many NaN gaps as possible
gpsX = iSlimsa['gpscoordinateX_x'].combine_first(iSlimsa['gpscoordinateX_y'])
gpsX = gpsX.combine_first(iSlimsa['gpscoordinateX_x'])
gpsY = iSlimsa['gpscoordinateY_x'].combine_first(iSlimsa['gpscoordinateY_y'])
gpsY = gpsY.combine_first(iSlimsa['gpscoordinateY_x'])

In [186]:
# Removing irrelevant variables
iSlimsb = iSlimsa.iloc[:,0:18]
iSlimsb['gpsX'] = gpsX
iSlimsb['gpsY'] = gpsY

In [187]:
# final merged dataset
iSlimsc = iSlimsb.drop(['gpscoordinateX_x', 'gpscoordinateY_x'], axis = 1)

In [188]:
# Throwing out observations without GPS coordinate
iSlimsd = iSlimsc.dropna(subset = ['gpsX', 'gpsY'])

In [189]:
# Will use codes: 2, 196, 201, 209
fc[fc['description'].str.contains('ight')].head()

,failcodeID,description
0,2,(Light out)
1,3,(Light off/on)
2,4,(Light dim)
10,12,(Trim tree from Light)
16,18,(Convert street light)


In [190]:
# Filtering by observations with the desired failure codes
iSlimse = iSlimsd[iSlimsd['finalresolutionID'].isin([2, 196, 201, 209])]

In [191]:
# Filtering by times of interest
resolv_t = (iSlimse['resolveddatetime'] > '2007-12-31') & (iSlimse['resolveddatetime'] < '2017-01-01') 
enter_t = (iSlimse['entereddate_x'] > '2007-12-31') & (iSlimse['entereddate_x'] < '2017-01-01')
iSlimsf = iSlimse[resolv_t & enter_t]

In [192]:
# Filtering out observations with excessively long completion / late completion times
iSlimsg = iSlimsf[(iSlimsf['daysToComplete'] <= 14) & (iSlims['daysLate'] <= 7)]
iSlimsg['finalresolutionID'].value_counts()

2      174716
196       488
209        79
201         5
Name: finalresolutionID, dtype: int64

In [193]:
# Dropping duplicate woID's
iSlims = iSlimsg.drop_duplicates(subset = ['woID'])

In [195]:
iSlims.to_excel('C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data/iSlims_final.xlsx')